In [14]:
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [15]:
env = GridworldEnv()

In [76]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    converged = False
    while not converged:
        Vprime = V.copy()
        max_change = 0
        for state, value in enumerate(V):
            # Bellman Equation:
            # V(s) = r + gamma * V(s')

            new_value = 0
            for action, action_probability in enumerate(policy[state]):
                exp_reward = 0
                exp_val = 0
                for prob, next_state, reward, done in env.P[state][action]:
                    exp_val += prob * V[next_state]
                    exp_reward += prob * reward

                expected_val_sprime = exp_reward + discount_factor * exp_val

                new_value += action_probability * expected_val_sprime

            max_change = max(abs(value - new_value), max_change)
            Vprime[state] = new_value
        if max_change < theta:
            converged = True
        V = Vprime

    return np.array(V)

In [77]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)

In [78]:
print(v)

[  0.         -13.99989315 -19.99984167 -21.99982282 -13.99989315
 -17.99986052 -19.99984273 -19.99984167 -19.99984167 -19.99984273
 -17.99986052 -13.99989315 -21.99982282 -19.99984167 -13.99989315
   0.        ]


In [79]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)